# Clean Additional Archival Metadata
Clean up additional metadata fields of archival material as recorded in the online archival catalog.  Another notebook (DataAnalysisPreparation.ipynb) will associate this metadata with the annotated descriptions so that any correlations between language, date, and geography metadata fields and gender bias labels can be studied.


**Metadata fields to clean:**

   [1.](#lang) language of material - `language`

   [2.](#date) date(s) of material - `unit_date`

   [3.](#geog) associated geographic locations of material - `geography`


**Files:**
* Input:  `CRC_units-grouped-by-fonds.csv`
* Output: `CRC_units-grouped-by-fonds_clean.csv`

***

Begin by importing the libraries we'll use:

In [3]:
import pandas as pd
import numpy as np
import os
import string
import re
from datetime import *
from dateutil.relativedelta import *
from dateutil.easter import *
from dateutil.rrule import *
from dateutil.parser import *

In [4]:
# Additional archival metadata to associate with labels and notes data
metadata = "../annot-prep/CRC_units-grouped-by-fonds.csv"

In [5]:
df_metadata = pd.read_csv(metadata, index_col=0)
print(df_metadata.shape)
df_metadata.head()

(1081, 6)


,eadid,unit_title,unit_identifier,unit_date,geography,language
0,Coll-1064,"['Papers of Professor Walter Ledermann', '1 (3...","['Coll-1064', 'Coll-1064/1', 'Coll-1064/2', 'C...","['1937-1954', '2 Feb 1937', '10 Feb 1937', '16...","['Edinburgh (Scotland)', 'Edinburgh (Scotland)...","['\n English\n ', 'English', 'English'..."
1,Coll-31,['Drawings from the Office of Sir Rowand Ander...,"['Coll-31', 'Coll-31/1', 'Coll-31/1/1', 'Coll-...","['1814-1924', '1874-1905', '1874-1879', '1874-...",[],"['\n English\n ', 'English', 'English'..."
2,Coll-51,['Papers of Sir Roderick Impey Murchison and h...,"['Coll-51', 'Coll-51/1', 'Coll-51/2', 'Coll-51...","['1771-1935', '1723-1935', '1770-1938', '1770-...","['Calcutta (India)', 'Europe', 'Scotland', 'Ta...","['\n English\n ', 'English', 'English'..."
3,Coll-204,"['Lecture Notes of John Robison', 'Introductio...","['Coll-204', 'Coll-204/1', 'Coll-204/2', 'Coll...","['c1779-c1801', 'c1779-c1801', 'c1804', 'c1802...","['Edinburgh (Scotland)', 'Glasgow Lanarkshire ...","['\n English\n ', 'English.', 'English..."
4,Coll-206,['Records of the Wernerian Natural History Soc...,"['Coll-206', 'Coll-206/1', 'Coll-206/1/1', 'Co...","['1808-1858', '12 January 1808-16 April 1858',...","['Edinburgh (Scotland)', 'Freiburg im Breisgau...","['\n English\n ', 'English', 'English'..."


Now let's clean up and remove duplicates from cells in the metadata fields for [language](#lang), [date](#date), and [geography](#geog)!

<a id="lang"></a>
## Column: language

In [6]:
language_list = list(df_metadata.language)
indeces = list(df_metadata.index)
lang_dict = dict.fromkeys(indeces)
# lang_dict  # Looks good

In [7]:
i = indeces[0]
maxI = indeces[-1]
while i <= maxI:
    languages = language_list[i]  #for languages in language_list:
    new_languages = []
    if languages.lower() == "multiple languages":
        new_languages += [languages]
    elif "Could be in Norman-French. French, Old" in languages:
        new_languages += ["Norman-French/French, Old"]
        languages = languages.replace("Could be in Norman-French. French, Old","")
    else:
        languages = languages.strip("[]")
        languages = languages.replace("'","")
        languages = languages.replace(".","")
        languages = languages.replace(";","")
        languages = languages.replace("\n","")
        languages = languages.replace("\\n","")
        languages = languages.replace("Material in","")
        languages = languages.replace("Material is in","")
        languages = languages.replace("This material is in","")
        languages = languages.replace("This material is mostly in","")
        languages = languages.replace("but also","")       # remove extra info about where particular language appears
        languages = languages.replace("Volume in","")      # remove extra info about where particular language appears
        languages = languages.replace("letter in","")      # remove extra info about where particular language appears
        languages = languages.replace("some material","")  # remove extra info about where particular language appears
        languages = languages.replace(" one ","")            # remove extra info about where particular language appears
        languages = languages.replace("One ","")            # remove extra info about where particular language appears
        languages = languages.replace("article","")            # remove extra info about where particular language appears
        languages = languages.replace("restEnglish","English")  # remove extra info about where particular language appears
        languages = languages.replace("or more","")        # remove extra info about where particular language appears
        languages = languages.replace(" in ","")           # remove extra info about where particular language appears
        languages = languages.replace(" with ","")           # remove extra info about where particular language appears
        languages = languages.replace("Telegram","")       # remove extra info about where particular language appears
        languages = languages.replace("One article in","")
        languages = languages.replace("rest in","")
        languages = languages.replace("Partly also in","") # remove extra info about language's presence
        languages = languages.replace("Other languages represented include","") # remove extra info about language's presence
        languages = languages.replace("[?]","")            # remove extra info about certainty of language's presence
        # reformat substring on next line so code below interprets the text as intended
        # (German already appears another time in group of languages so can remove from this substring)
        languages = languages.replace("Mostly English  German","Mostly English")  
        languages = languages.replace("dilect","dialect")  # fix typo
        languages = languages.replace("Dilect","Dialect")  # fix typo
        languages = languages.replace("JapaneseEnglish","Japanese English")  # fix typo
        languages = languages.replace("Egnlish","English")  # fix typo
        languages = languages.replace("MostlyEnglish","Mostly English")  # fix typo
        languages = languages.replace("English English shorth","English")  # CONFIRM THAT THIS IS TYPO!
        languages = languages.replace('code "tsw" not foundISO 639-2 list',"")
        if " and " in languages:
            sub_lists = languages.split(" and ")
            unique_languages = []
            for sub_list in sub_lists:
                sub_langs = sub_list.split(", ")
                for sub_l in sub_langs:
                    unique_languages += [sub_l]
            unique_languages = set(unique_languages)
        else:
            languages = languages.split(", ")
            unique_languages = set(languages)
#         print("unique_languages:",unique_languages)
        for l in unique_languages:
#             print("l:",l)
            l = l.strip()
            if (l.lower() == "multiple languages") and (l not in new_languages):
                new_languages += [l]
            elif "," in l:
                sub_l_list = l.split(",")
                for sub_l in sub_l_list:
                    if (sub_l not in new_languages) and (len(sub_l) != 0):
                        new_languages += [sub_l]
            elif (" " in l) and ("languages" not in l) and ("translation" not in l) and ("Medieval" not in l) and ("Anglo" not in l) and ("mainly" not in l) and ("Mainly" not in l) and ("mostly" not in l) and ("Mostly" not in l) and ("or" not in l) and ("dialect" not in l) and ("gaelic" not in l) and ("Dialect" not in l) and ("Gaelic" not in l):
                    sub_l_list = l.split(" ")
#                     print("sub_l_list:",sub_l_list)
                    for sub_l in sub_l_list:
                        if sub_l == "(Ancient)":
                            first_sub = new_languages[-1]
                            new_languages = new_languages[:-2]
                            new_languages += [first_sub + " " + sub_l]
                        elif "," in sub_l:
                            sub_sub_l_list = sub_l.split(",")
                            for sub_sub_l in sub_sub_l_list:
                                if (sub_sub_l not in new_languages) and (len(sub_sub_l) != 0):
                                    new_languages += [sub_sub_l]
                        elif sub_l not in new_languages:
                            if len(sub_l) != 0:
                                new_languages += [sub_l]
            else:
                if (l not in new_languages) and (len(l) != 0):
                    new_languages += [l]
    if "Englishand" in new_languages:  # fix typo (English already in list)
        new_languages = new_languages.remove("Englishand")
#     print("new_languages:",new_languages)  # Looks good!
    lang_dict[i] = new_languages
    i += 1
print(lang_dict)  # Looks good!

{0: ['English'], 1: ['English'], 2: ['English'], 3: ['English', 'French', 'Latin', 'Russian'], 4: ['English'], 5: ['English', 'French', 'Latin', 'Mostly English'], 6: ['English'], 7: ['English'], 8: ['English'], 9: ['English', 'Latin'], 10: ['English'], 11: [], 12: ['Latin', 'English'], 13: ['English'], 14: ['English', 'Gaelic or Scottish Gaelic'], 15: ['English'], 16: ['English'], 17: ['English'], 18: ['English'], 19: ['Multiple languages'], 20: ['English'], 21: ['English'], 22: ['English'], 23: ['English', 'Latin'], 24: ['English'], 25: ['English'], 26: [], 27: ['English'], 28: ['English'], 29: ['English'], 30: ['English'], 31: ['English'], 32: ['English'], 33: ['English'], 34: ['English'], 35: ['English'], 36: ['English'], 37: ['English'], 38: [], 39: ['English'], 40: ['English'], 41: [], 42: ['Latin'], 43: ['English', 'Latin'], 44: ['French', 'English'], 45: ['English'], 46: ['English'], 47: ['English'], 48: ['English'], 49: ['English'], 50: ['English'], 51: ['English', 'French'], 

In [9]:
df_metadata = df_metadata.assign(unique_language = lang_dict.values())
df_metadata.head() # Looks good
# df_metadata.to_csv("CRC_units-grouped-by-fonds_clean.csv")

,eadid,unit_title,unit_identifier,unit_date,geography,language,unique_language
0,Coll-1064,"['Papers of Professor Walter Ledermann', '1 (3...","['Coll-1064', 'Coll-1064/1', 'Coll-1064/2', 'C...","['1937-1954', '2 Feb 1937', '10 Feb 1937', '16...","['Edinburgh (Scotland)', 'Edinburgh (Scotland)...","['\n English\n ', 'English', 'English'...",[English]
1,Coll-31,['Drawings from the Office of Sir Rowand Ander...,"['Coll-31', 'Coll-31/1', 'Coll-31/1/1', 'Coll-...","['1814-1924', '1874-1905', '1874-1879', '1874-...",[],"['\n English\n ', 'English', 'English'...",[English]
2,Coll-51,['Papers of Sir Roderick Impey Murchison and h...,"['Coll-51', 'Coll-51/1', 'Coll-51/2', 'Coll-51...","['1771-1935', '1723-1935', '1770-1938', '1770-...","['Calcutta (India)', 'Europe', 'Scotland', 'Ta...","['\n English\n ', 'English', 'English'...",[English]
3,Coll-204,"['Lecture Notes of John Robison', 'Introductio...","['Coll-204', 'Coll-204/1', 'Coll-204/2', 'Coll...","['c1779-c1801', 'c1779-c1801', 'c1804', 'c1802...","['Edinburgh (Scotland)', 'Glasgow Lanarkshire ...","['\n English\n ', 'English.', 'English...","[English, French, Latin, Russian]"
4,Coll-206,['Records of the Wernerian Natural History Soc...,"['Coll-206', 'Coll-206/1', 'Coll-206/1/1', 'Co...","['1808-1858', '12 January 1808-16 April 1858',...","['Edinburgh (Scotland)', 'Freiburg im Breisgau...","['\n English\n ', 'English', 'English'...",[English]


<a id="date"></a>
## Column: unit_date

In [10]:
# df_metadata = pd.read_csv("CRC_units-grouped-by-fonds_clean.csv", index_col=0) # load the metadata with clean language column
unit_date_list = list(df_metadata.unit_date)
indeces = list(df_metadata.index)
date_dict = dict.fromkeys(indeces)
# df_metadata.head()  # Looks good

In [11]:
# INPUT: dictionary with keys as metadata DataFrame indeces and values empty, and list of unit_dates
# OUTPUT: dictionary with keys as indeces and values as dates with 'YY-?' converted to 'YYYY - YYYY'
def replaceDashQuestionMark(date_dict, unit_date_list):
    indeces = date_dict.keys()
    for i in indeces:
        given_dates = unit_date_list[i]
        if "-?" in given_dates:
            to_replace = re.findall("\d{3}-+\?", given_dates)
            for d in to_replace:
                digits = re.match("\d{3}", d)[0]
                start_year = digits+"0"
                end_year = digits+"9"
                new_d = start_year + " - " + end_year
                given_dates.replace(d, new_d)
        date_dict[i] = given_dates
    return date_dict

# INPUT: date dictionary from previous function and lists below
punct_uncert = ["tbc", "circa ", "circa", "approximately", "probably", "(", ")", "?"]
days_of_week = ["monday", "tuesday", "wednesday", "thursday", "friday", "saturday", "sunday"]
unknown = ["undated", "no date", "n.d.", "nd" "xxxx", "year unknown"]
punct_newline = ["[","]","(",")",".","'","\\n"]
# OUTPUT: date dictionary with uncertainty and punctuation removed and values lowercased, and
#         boolean for whether BCE dates exist
def fixPunctAndUncertainty(date_dict, to_remove=punct_uncert+days_of_week+punct_newline, no_date=unknown):
    indeces = date_dict.keys()
#     have_bc = False  # Check to see whether BCE dates exist - NONE EXISTED
    for i in indeces:
        given_dates = date_dict[i]
        given_dates = given_dates.lower()
#         if "bc" in given_dates:
#             have_bc = True
        given_dates = given_dates.replace("/", " - ")
        for x in no_date:
            given_dates = given_dates.replace(x, "0000")
        for y in to_remove:
            given_dates = given_dates.replace(y,"")
        # Remove abbreviations for "circa" ("c", "c.", "ca.") without removing c from any month's names
        circas = re.findall("(?<![ero])c[a]?\.*[ ]*\d{4}", given_dates)
        for d in circas:
            new_d = re.findall("\d{4}", d)[0]
            given_dates = given_dates.replace(d, new_d)
        # Replace "s" with date range ("1900s" --> "1900 - 1999", "1830s-1840s" --> "1830 - 1849")
        s_range_start = re.findall("\d{4}s(?=-)", given_dates)
        s_range_end = re.findall("(?<=-)\d{4}s", given_dates)
        s_range = re.findall("(?<!-)\d{4}s(?!-)", given_dates)
        for d in s_range_start:
            new_d = re.findall("\d{4}", d)[0]
            given_dates = given_dates.replace((d+"-"), (new_d+" -"))
        for d in s_range_end:
            new_d = re.findall("\d{4}", d)[0]
            given_dates = given_dates.replace(("-"+d), ("- "+new_d))
        for d in s_range:
            new_d_start = int(re.findall("\d{4}", d)[0])
            if new_d_start % 100 == 0:
                new_d = str(new_d_start)+" - "+str(new_d_start+99)
            else:
                new_d = str(new_d_start)+" - "+str(new_d_start+9)
            given_dates = given_dates.replace(d, new_d)
        date_dict[i] = given_dates
    return date_dict #, have_bc

In [12]:
date_dict = replaceDashQuestionMark(date_dict, unit_date_list)
date_dict = fixPunctAndUncertainty(date_dict)
# print(date_dict[1080])   # check for undated removal, month removal, \n removal s ranges,  - DONE!
# print(date_dict[101])   # check for circa abv. and [] removal - DONE!

In [13]:
# INPUT: date dictionary with keys as ineces and values as unit dates from metadata
# OUTPUT: date dictionary where any values with a season name in them are replaced with a year range
#         in the format YYYY-MM-D - YYYY-MM-DD
def fixSeasons(date_dict):
    f_months_days = {"startm":"9", "endm":"11", "startd":"1", "endd":"30"}
    w_months_days = {"startm":"12", "endm":"2", "startd":"1", "endd":"28"}  # Ignoring leap years
    sp_months_days = {"startm":"3", "endm":"5", "startd":"1", "endd":"31"}
    su_months_days = {"startm":"6", "endm":"8", "startd":"1", "endd":"31"}
    season_months_days = {"fall":f_months_days, "autumn":f_months_days, "winter":w_months_days, 
               "spring":sp_months_days, "summer":su_months_days}
    seasons = season_months_days.keys()
    indeces = date_dict.keys()
    for i in indeces:
        given_dates = date_dict[i]
        for s in seasons:
            if s in given_dates:
                season_dates = re.findall(s+"\d{4}",given_dates)
                for d in season_dates:
                    y = re.findall("\d{4}",d)[0]
                    start_m = season_months_days[s][startm]
                    end_m = season_months_days[s][endm]
                    start_s = season_months_days[s][startd]
                    end_d = season_months_days[s][endd]
                    new_date = y+"-"+start_m+"-"+start_d+" - "+y+"-"+end_m+"-"+end_d
                    given_dates = given_dates.replace(d,new_date)
        date_dict[i] = given_dates
    return date_dict

In [14]:
date_dict = fixSeasons(date_dict)

In [15]:
### No BC[E] dates found so this function is not needed
# def findBeforeAfterBCE(date_dict):
#     before_list, after_list, bce_list = [], [], []
#     indeces = date_dict.keys()
#     for i in indeces:
#         given_dates = date_dict[i]
#         if "before" in given_dates:
#             before_list += [given_dates]
#         if "after" in given_dates:         
#             after_list += [given_dates]
#         if "bc" in given_dates:            # These were within "tbc" so removed in previous function
#             bce_list += [given_dates]
#     return before_list, after_list, bce_list

# Formats to replace: before 1881-1916, before 1969, first half of the 20th century before 1944,
#                     early 20th century, before 2005. date of original unknown.
# Replace all with YYYY - YYYY
def fixBefore(date_dict):
    indeces = date_dict.keys()
    for i in indeces:
        given_dates = date_dict[i]
        if "first half of" and "before" in given_dates:
            re_string = "(first half of the \d{1,2}[a-z]{0,2} century (before|pre) \d{4})"
            to_replace = re.findall(re_string, given_dates)
            for d in to_replace:
                d = d[0]
                century = re.findall("\d{1,2}[a-z]{0,2} century", d)[0]
                start_year_digits = int(re.findall("\d{1,2}", century)[0]) - 1
                end_year = int(re.findall("\d{4}", d)[0]) - 1
                new_d = str(start_year_digits)+"00 - "+str(end_year)
                given_dates = given_dates.replace(d, new_d)
        if "early" and "century" and ("before" or "pre") in given_dates:
            to_replace = re.findall("(early \d{1,2}\[a-z]{0,2} century (before|pre) \d{4})", given_dates)
            for d in to_replace:
                century = re.findall("\d{1,2}[a-z]{0,2}(?=( century))", d)[0][0]
                start_year_digits = int(century) - 1
                before_year = re.findall("(?<=(ore |pre ))\d{4}", d)
                end_year = int(before_year) - 1
                new_d = str(start_year_digits)+"00 - "+str(end_year)
                given_dates = given_dates.replace(d, new_d)
        if (("before" or "pre") in given_dates): # and ("century" not in given_dates):
            to_replace = re.findall("(?<=(ore |pre ))\d{4}-\d{4}", given_dates)  # find year ranges
            for d in to_replace:
                before_year_list = re.findall("\d{4}-", d)
                if len(before_year_list) > 0:
                    before_year = before_year_list[0]
                    start_year = int(before_year[:-1]) - 1
                end_year_list = re.findall("-\d{4}", d)
                if len(end_year_list) > 0:
                    end_year = int(end_year_list[0][1:]) - 1
                    new_d = str(start_year)+" - "+str(end_year)
                    given_dates = given_dates.replace(d, new_d)
            to_replace = re.findall("(?<=(ore |pre ))\d{4}(?!-)", given_dates)  # find single years
            for d in to_replace:
                before_year_list = re.findall("\d{4}", d)
                if len(before_year_list) > 0:
                    before_year = before_year_list[0]
                    start_year = int(before_year) - 50
                    end_year = int(before_year) - 1
                    new_d = str(start_year)+" - "+str(end_year)
                    given_dates = given_dates.replace(d, new_d)
        date_dict[i] = given_dates
    return date_dict

# # Only 'after 2nd july 1550'
# Add 1 day for start date and add 50 years from then for end date 
def fixAfter(date_dict):
    indeces = date_dict.keys()
    for i in indeces:
        given_dates = date_dict[i]
        if "after 2nd july 1550" in given_dates:
            given_dates = given_dates.replace("after 2nd july 1550", "1550-07-03 - 1600-07-03")
        date_dict[i] = given_dates
    return date_dict

In [16]:
date_dict = fixBefore(date_dict)
date_dict = fixAfter(date_dict)
# before, after, bce = findBeforeAfterBCE(date_dict) - NO BC[E] DATES IN DATASET
# print(len(before), len(after), len(bce))
# print("Before:\n",before)  # Formats above findBefore(...) function in previous cell
# print("\nAfter:\n",after)  # Only 'after 2nd july 1550'
# print("\BCE:\n",bce)       # No BCE dates

In [17]:
century_words = ["first", "second", "third", "fourth", "fifth", "sixth", "seventh", "eighth", "ninth", "tenth"]

# Replace century ranges and multi-century estimates with year ranges in the format: YYYY - YYYY
def replaceCenturies(date_dict, century_words=century_words):
    indeces = date_dict.keys()
    for i in indeces:
        given_dates = date_dict[i]
        if "centuries" or "century" in given_dates:
            for c in century_words:
                if c in given_dates:
                    two_centuries = "[a-z]{5,7} (or|to|and) [a-z]{5,7} (centuries|century)"
                    d_list = re.findall(two_centuries, given_dates)
                    for d in d_list:
                        start_century = re.findall("[a-z]{5,7}(?= (or|to|and))", d)[0]
                        end_century = re.findall("(?<=( or| to|and) )[a-z]{5,7}", d)[0]
                        if start_century == "first":
                             start_year = str(century_words.index("first"))+"0"
                        else:
                            start_year = str(century_words.index(start_century))+"00"
                        end_year = str(century_words.index(end_century))+"99"
                        new_d = str(start_year)+" - "+str(end_year)
                        given_dates = given_dates.replace(d, new_d)
        date_dict[i] = given_dates
    return date_dict

# Replace all occurrences of a singular century with date ranges in the format: YYYY - YYYY 
def replaceCentury(date_dict, century_words=century_words):
    indeces = date_dict.keys()
    for i in indeces:
        given_dates = date_dict[i]
        if "century" in given_dates:
            for c in century_words:
                # If the century number is written as a word, not digits
                if c in given_dates:
                    d_list = re.findall(c+" century", given_dates)
                    for d in d_list:
                        if c == "first":
                             start_year = str(century_words.index(c))+"0"
                        else:
                            start_year = str(century_words.index(c))+"00"
                        end_year = int(start_year) + 99
                        new_d = str(start_year)+" - "+str(end_year)
                        given_dates = given_dates.replace(d, new_d)
                # If the century number is written as digits, not a word
                else:
                    early = re.findall("early \d{2}[a-z]{2} century", given_dates)
                    late = re.findall("late \d{2}[a-z]{2} century", given_dates)
                    if len(early) > 0:
                        for d in early:
                            digits = re.findall("\d{2}", d)
                            if len(digits) == 0:
                                digits = re.findall("\d{1}", d)
                            start_year = (int(digits[0]) - 1)*100
                            end_year = start_year + 49
                            new_d = str(start_year)+" - "+str(end_year)
                            given_dates = given_dates.replace(d, new_d)
                    if len(late) > 0:
                        for d in late:
                            digits = re.findall("\d{2}", d)
                            if len(digits) == 0:
                                digits = re.findall("\d{1}", d)
                            start_year = ((int(digits[0]) - 1)*100)+50
                            end_year = start_year + 49
                            new_d = str(start_year)+" - "+str(end_year)
                            given_dates = given_dates.replace(d, new_d)
                    century = re.findall("\d{1,2}[a-z]{0,2} century", given_dates)
                    for d in century:
                        digits = re.findall("\d{2}", d)
                        if len(digits) == 0:
                            digits = re.findall("\d{1}", d)
                        start_year = (int(digits[0]) - 1)*100
                        end_year = start_year + 99
                        new_d = str(start_year)+" - "+str(end_year)
                        given_dates = given_dates.replace(d, new_d)
            date_dict[i] = given_dates
    return date_dict

In [18]:
date_dict = replaceCenturies(date_dict)
date_dict = replaceCentury(date_dict)
print(date_dict[10])  # check for early 19th century conversion - DONE!

1790-1850, 1800 - 1849, 1797, 1798, 1798, 1799, 1799, 1802, 1811, 1811, 1813, march 1816, 1819, 1792-1793


In [19]:
month_days_dict = {"january":31, "february":28, "march":31, "april":30, "may":31, "june":30,
                  "july":31, "august":31, "september":30, "october":31, "november":30, "december":31}
month_list = month_days_dict.keys()
# NOTE: the function below assumes date formats where day precedes month!
# ALSO NOTE: leap years ignored so latest day in february is always 28
def addDayIfNone(date_dict, months=month_list, month_days=month_days_dict):
    indeces = date_dict.keys()
    for i in indeces:
        given_dates = date_dict[i]
        dates_list = given_dates.split(", ")
        new_dates = []
        for d in dates_list:
            word = re.findall("^[a-z]+", d)
            if (len(word) > 0) and (word[0] in month_list):
                start_date = "1"+" "+d
                end_date = str(month_days_dict[word[0]])+" "+d
                new_d = start_date + " - " + end_date
                new_dates += [new_d]
            else:
                new_dates += [d]
        date_dict[i] = new_dates
    return date_dict

def formatDate(input_date, start, months=month_list, month_days=month_days_dict):
    not_processed = []
    # Check for unknown dates:
    if "0000" in input_date:
        return "0000-00-00", not_processed
    else:
        # Check for 1[0] jan[uary] 2222 format
        full_date = re.findall("\d{1,2} [a-z]{3,9} \d{4}", input_date)
        if len(full_date) > 0:
            try:
                output_date = parse(full_date[0])
            except ValueError:
                not_processed += [full_date[0]]
                output_date = "0000-00-00"
        else:
            # Check for jan[uary] 2222 format
            partial_date = re.findall("[a-z]{3,9} \d{4}",input_date)
            if len(partial_date) > 0:
                if start:
                    d = "01"
                else:
                    m = re.findall("[a-z]{3,9}",partial_date[0])[0]
                    for month in months:
                        # allow for abbreviations (so "aug" will match "august", for example)
                        if m in month:
                            d = str(month_days[month])
                try:
                    output_date = parse(d+" "+partial_date[0])
                except ValueError:
                    not_processed += [partial_date[0]]
                    output_date = "0000-00-00"
            else:
                # Check for 2222 format
                year_list = re.findall("\d{4}",input_date)
                if len(year_list) > 0:
                    if start:
                        output_date = datetime(int(year_list[0]),1,1)
                    else:
                        output_date = datetime(int(year_list[0]),12,31)
                else:
                    not_processed += [input_date]
                    output_date = "0000-00-00"
        return output_date, not_processed
            
# INPUT: dictionary of indeces as keys and list of strings (date or date range) as values
# OUTPUT: dictionary where values' dates are all a range in the format: YYYY-MM-DD - YYYY-MM-DD
def formatDateRanges(date_dict):
    indeces = date_dict.keys()
    start, end = False, False
    not_processed_start, not_processed_end, not_processed_start_date, not_processed = [], [], [], []
    for i in indeces:
        given_dates = date_dict[i]
        new_dates = []
        for d in given_dates:
            # Check for date range (including June-July 1944 and 26 January 1907-1925 formats)
            if "-" in d:
                d_pair = d.split("-")
                # if the date isn't already in the format YYYY-MM-DD - YYYY-MM-DD
                if len(d_pair) == 2:
                    d_pair0 = d_pair[0].strip()  # remove leading and trailing whitespace
                    d_pair1 = d_pair[1].strip()  # remove leading and trailing whitespace
                    # If year only in second of pair, should add to first of pair
                    if (len(re.findall("\d{4}",d_pair0)) == 0) and (len(re.findall("\d{4}",d_pair1)) == 1):
                        d_pair0 = d_pair0 + " " + re.findall("\d{4}",d_pair1)[0]
                    start_input, not_processed_start = formatDate(d_pair0, True)
                    not_processed += [not_processed_start]
                    start = re.findall("\d{4}-\d{2}-\d{2}", str(start_input))[0]
                    end_input, not_processed_end= formatDate(d_pair1, False)
                    not_processed += [not_processed_end]
                    end = re.findall("\d{4}-\d{2}-\d{2}", str(end_input))[0]
            # Check for single date (full or partial)
            else:
                if "0000" in d:
                    start, end = "0000-00-00", "0000-00-00"
                else:
                    try:
                        start_date, not_processed_start_date = formatDate(d, True)
                        not_processed += [not_processed_start_date]
                        if start_date != "0000-00-00":
                            drange = list(rrule(freq=YEARLY, count=2, dtstart=start_date))
                            start = re.findall("\d{4}-\d{2}-\d{2}", str(drange[0]))[0]
                            end = re.findall("\d{4}-\d{2}-\d{2}", str(drange[1]+relativedelta(days=-1)))[0]
                        else:
                            start, end = start, end = "0000-00-00", "0000-00-00"
                    except ValueError:
                        not_processed += [d]
            if start and end:
                new_d = str(start) + " - " + str(end)
                new_dates += [new_d]  # add date formatted as YYYY-MM-DD - YYYY-MM-DD
            else:
                new_dates += [d]      # if date already formatted properly, keep as is
        date_dict[i] = new_dates
    return date_dict, not_processed

In [20]:
# century_words = ["first", "second", "third", "fourth", "fifth", "sixth", "seventh", "eighth", "ninth", "tenth"]
# # century_words.index("first")
# len_c_w = [len(c) for c in century_words]
# print(min(len_c_w), max(len_c_w))
# print(date_dict[10].split(", "))
# month_list_lens = [len(x) for x in month_list]
# print(min(month_list_lens),max(month_list_lens))
# print(date_dict[0])
# x = "pre 1940"
# x[-4:]

date_dict = addDayIfNone(date_dict)
date_dict, not_processed = formatDateRanges(date_dict)

print("\n date dictionary sample")
print(date_dict[10])  # check march now has range of 1 - 31 - DONE!
print("\n not processed")
print(not_processed)


 date dictionary sample
['1790-01-01 - 1850-12-31', '1800-01-01 - 1849-12-31', '1797-01-01 - 1797-12-31', '1798-01-01 - 1798-12-31', '1798-01-01 - 1798-12-31', '1799-01-01 - 1799-12-31', '1799-01-01 - 1799-12-31', '1802-01-01 - 1802-12-31', '1811-01-01 - 1811-12-31', '1811-01-01 - 1811-12-31', '1813-01-01 - 1813-12-31', '1816-03-01 - 1816-03-31', '1819-01-01 - 1819-12-31', '1792-01-01 - 1793-12-31']

 not processed
[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [],

In [21]:
##########################
# Not Processed:
##########################
# "before 1795"
# "mid 1900"
# "the 1900"
# "pre 1940"
# "about 1200"
# "19"  # count or only if has ad before or after?
# "07"  # count or only if has ad before or after?
# "ad 17"
# "ad316"
# "599 ad"
# "200"
# "1st"
# "6th centuries"
# "late 1950"
# "early 1950"
# "caugust 1855"
# "late 19th"
# "26 sepember 1917"
# "january 1961"
# "30 feb 1940"
# "autumn 1971"
# "1 september 196"  # was there a square bracket next?
# ['25 juni 1949'], ['1 juli 1949'],['9 mai 1946'],['8 marz 1948']
# "1 novmeber 1976"

# "not indicated"
# "xxxx"
# "unknown"
# "not known"

I'm going to call this good enough for now.  I can always go back and account for the various formats of date values in the above cell that weren't processed.

In [22]:
#### DONE IN FUNCTIONS ABOVE #########################################################################
# Replace "196-?" with 1960 - 1969
# Remove "c", "c ", "c.", "c. ", "ca.", "ca" "circa", "approximately", "probably", "[]", "()", "Between", "between",, 
#        "YYYYs", "YYYYs-YYYYs" and "?" indicating uncertainty
# Remove "'\" and "\'" and "nd"
# Remove "Monday", "Tuesday", etc. (days of the week)
# Replace "No date" and "no date" and "undated" and "n.d." and "xxxx" and "year unknown" with 
#         "0000" (as start and end date)
# Replace "/" with " - " and interpret as range of dates
# Replace "Summer YYYY" with YYYY-05-01 - YYYY-08-31, etc. (consider all seasons)
# Search for "B/before" and "A/after" and manually replace...
##### 'first half of the 20th century (before 1944)' with 1900 - 1944
#####  Replace "before 1777" or "Before 1777" with 1700-1776, etc.
# Replace "20th Century" with "1900 - 1999," "late 19th century", "early...", 
#         '1st - 6th centuries','Second century AD','173 AD', 'AD. 115-116',
#         'Second or early third century AD.',  etc.
#####################################################################################################
# Replace month names with digits --> YYYY-MM-DD
# Remove " (creation of notes and research material themselves)", " (date of source material)", 
#        " (date of notes themselves)" ... basically all other text that's not month name or digit!
# Replace all ranges with a "-" to have spaces around the dash: " - "
# # # Some have no space, some two, some have one space such as: "1 march 2000- 29 may 2000" 

# Create unit_start_date and unit_end_date columns (if range not given, put same date in both columns)

# Represent all dates in datetime format range YYYY-MM-DD - YYY-M-D
# Translate from the following formats:
# 'October 1851'      --> change to 1851-10-1 - 1851-10-31
# '11July 1792'
# '06 November 1788'
# '4 October 1911'
# '1841'              --> change to 1841-01-1 - 1841-31-12
# '1886-1913'
#  '1891 - 1928'
# '1870s-1930s'       ---> change to 1870-1939
# '1940-50s'          ---> change to 1940-1959
# '18th century-1863' --> change to 1700-1863
# '01 November 1867-05 May 1982'
# '23 December 1912-6 February 1928'
# '6 Feb 1789 – 26 Dec 1789'
# 'June 1994 - January 2000'  --> change to 1994-06-1 - 2000-01-31
# '1984-11-01 - 1984-11-01'
# '1997-02 - 2002-05'
# 'June-July 1944'
# ''May - June 1965''
# '26 January 1907-1925'
# '1st - 6th centuries','Second century AD','173 AD', 'AD. 115-116','Second or early third century AD.',17th -18th century

In [23]:
df_metadata = df_metadata.assign(unique_date = date_dict.values())
df_metadata.head() # Looks good

,eadid,unit_title,unit_identifier,unit_date,geography,language,unique_language,unique_date
0,Coll-1064,"['Papers of Professor Walter Ledermann', '1 (3...","['Coll-1064', 'Coll-1064/1', 'Coll-1064/2', 'C...","['1937-1954', '2 Feb 1937', '10 Feb 1937', '16...","['Edinburgh (Scotland)', 'Edinburgh (Scotland)...","['\n English\n ', 'English', 'English'...",[English],"[1937-01-01 - 1954-12-31, 1937-02-02 - 1938-02..."
1,Coll-31,['Drawings from the Office of Sir Rowand Ander...,"['Coll-31', 'Coll-31/1', 'Coll-31/1/1', 'Coll-...","['1814-1924', '1874-1905', '1874-1879', '1874-...",[],"['\n English\n ', 'English', 'English'...",[English],"[1814-01-01 - 1924-12-31, 1874-01-01 - 1905-12..."
2,Coll-51,['Papers of Sir Roderick Impey Murchison and h...,"['Coll-51', 'Coll-51/1', 'Coll-51/2', 'Coll-51...","['1771-1935', '1723-1935', '1770-1938', '1770-...","['Calcutta (India)', 'Europe', 'Scotland', 'Ta...","['\n English\n ', 'English', 'English'...",[English],"[1771-01-01 - 1935-12-31, 1723-01-01 - 1935-12..."
3,Coll-204,"['Lecture Notes of John Robison', 'Introductio...","['Coll-204', 'Coll-204/1', 'Coll-204/2', 'Coll...","['c1779-c1801', 'c1779-c1801', 'c1804', 'c1802...","['Edinburgh (Scotland)', 'Glasgow Lanarkshire ...","['\n English\n ', 'English.', 'English...","[English, French, Latin, Russian]","[1779-01-01 - 1801-12-31, 1779-01-01 - 1801-12..."
4,Coll-206,['Records of the Wernerian Natural History Soc...,"['Coll-206', 'Coll-206/1', 'Coll-206/1/1', 'Co...","['1808-1858', '12 January 1808-16 April 1858',...","['Edinburgh (Scotland)', 'Freiburg im Breisgau...","['\n English\n ', 'English', 'English'...",[English],"[1808-01-01 - 1858-12-31, 1808-01-12 - 1858-04..."


In [ ]:
# df_metadata.to_csv("CRC_units-grouped-by-fonds_clean.csv")

<a id="geog"></a>
## Column: geography

In [24]:
# df_metadata = pd.read_csv("CRC_units-grouped-by-fonds_clean.csv", index_col=0) # load the metadata with clean language column
geography_list = list(df_metadata.geography)
indeces = list(df_metadata.index)
geog_dict = dict.fromkeys(indeces)
df_metadata.head()  # Looks good

,eadid,unit_title,unit_identifier,unit_date,geography,language,unique_language,unique_date
0,Coll-1064,"['Papers of Professor Walter Ledermann', '1 (3...","['Coll-1064', 'Coll-1064/1', 'Coll-1064/2', 'C...","['1937-1954', '2 Feb 1937', '10 Feb 1937', '16...","['Edinburgh (Scotland)', 'Edinburgh (Scotland)...","['\n English\n ', 'English', 'English'...",[English],"[1937-01-01 - 1954-12-31, 1937-02-02 - 1938-02..."
1,Coll-31,['Drawings from the Office of Sir Rowand Ander...,"['Coll-31', 'Coll-31/1', 'Coll-31/1/1', 'Coll-...","['1814-1924', '1874-1905', '1874-1879', '1874-...",[],"['\n English\n ', 'English', 'English'...",[English],"[1814-01-01 - 1924-12-31, 1874-01-01 - 1905-12..."
2,Coll-51,['Papers of Sir Roderick Impey Murchison and h...,"['Coll-51', 'Coll-51/1', 'Coll-51/2', 'Coll-51...","['1771-1935', '1723-1935', '1770-1938', '1770-...","['Calcutta (India)', 'Europe', 'Scotland', 'Ta...","['\n English\n ', 'English', 'English'...",[English],"[1771-01-01 - 1935-12-31, 1723-01-01 - 1935-12..."
3,Coll-204,"['Lecture Notes of John Robison', 'Introductio...","['Coll-204', 'Coll-204/1', 'Coll-204/2', 'Coll...","['c1779-c1801', 'c1779-c1801', 'c1804', 'c1802...","['Edinburgh (Scotland)', 'Glasgow Lanarkshire ...","['\n English\n ', 'English.', 'English...","[English, French, Latin, Russian]","[1779-01-01 - 1801-12-31, 1779-01-01 - 1801-12..."
4,Coll-206,['Records of the Wernerian Natural History Soc...,"['Coll-206', 'Coll-206/1', 'Coll-206/1/1', 'Co...","['1808-1858', '12 January 1808-16 April 1858',...","['Edinburgh (Scotland)', 'Freiburg im Breisgau...","['\n English\n ', 'English', 'English'...",[English],"[1808-01-01 - 1858-12-31, 1808-01-12 - 1858-04..."


I'm not going to worry about cleaning up the `geography` column for now, other than getting rid of space characters, such as `\n`, and repeated values within a row:

In [25]:
geogs = list(df_metadata.geography)
i = 0
for value in geogs:
    new_value = value.replace('\n','')
    new_value = new_value.strip('[')
    new_value = new_value.strip(']')
    new_value = new_value.replace("'","")
    new_value = new_value.strip()  # get rid of any leading and trailing white space
    new_value_list = new_value.split(', ')
    unique_new_values = list(set(new_value_list))
    geog_dict[i] = unique_new_values
    i += 1

In [26]:
k = list(geog_dict.keys())[0]
print(geog_dict[0])

['Edinburgh (Scotland)', 'London (England)', 'Glenapp Ayrshire Scotland', 'St Andrews (Scotland)']


In [27]:
df_metadata = df_metadata.assign(unique_geography = geog_dict.values())
df_metadata.head() # Looks good

,eadid,unit_title,unit_identifier,unit_date,geography,language,unique_language,unique_date,unique_geography
0,Coll-1064,"['Papers of Professor Walter Ledermann', '1 (3...","['Coll-1064', 'Coll-1064/1', 'Coll-1064/2', 'C...","['1937-1954', '2 Feb 1937', '10 Feb 1937', '16...","['Edinburgh (Scotland)', 'Edinburgh (Scotland)...","['\n English\n ', 'English', 'English'...",[English],"[1937-01-01 - 1954-12-31, 1937-02-02 - 1938-02...","[Edinburgh (Scotland), London (England), Glena..."
1,Coll-31,['Drawings from the Office of Sir Rowand Ander...,"['Coll-31', 'Coll-31/1', 'Coll-31/1/1', 'Coll-...","['1814-1924', '1874-1905', '1874-1879', '1874-...",[],"['\n English\n ', 'English', 'English'...",[English],"[1814-01-01 - 1924-12-31, 1874-01-01 - 1905-12...",[]
2,Coll-51,['Papers of Sir Roderick Impey Murchison and h...,"['Coll-51', 'Coll-51/1', 'Coll-51/2', 'Coll-51...","['1771-1935', '1723-1935', '1770-1938', '1770-...","['Calcutta (India)', 'Europe', 'Scotland', 'Ta...","['\n English\n ', 'English', 'English'...",[English],"[1771-01-01 - 1935-12-31, 1723-01-01 - 1935-12...","[Tarradale (Scotland), Europe, Scotland, Calcu..."
3,Coll-204,"['Lecture Notes of John Robison', 'Introductio...","['Coll-204', 'Coll-204/1', 'Coll-204/2', 'Coll...","['c1779-c1801', 'c1779-c1801', 'c1804', 'c1802...","['Edinburgh (Scotland)', 'Glasgow Lanarkshire ...","['\n English\n ', 'English.', 'English...","[English, French, Latin, Russian]","[1779-01-01 - 1801-12-31, 1779-01-01 - 1801-12...","[Edinburgh (Scotland), Glasgow Lanarkshire Sco..."
4,Coll-206,['Records of the Wernerian Natural History Soc...,"['Coll-206', 'Coll-206/1', 'Coll-206/1/1', 'Co...","['1808-1858', '12 January 1808-16 April 1858',...","['Edinburgh (Scotland)', 'Freiburg im Breisgau...","['\n English\n ', 'English', 'English'...",[English],"[1808-01-01 - 1858-12-31, 1808-01-12 - 1858-04...","[Edinburgh (Scotland), Freiburg im Breisgau (G..."


In [28]:
# Remove the columns that now have a `unique_` version:
df_metadata.drop(['unit_date', 'geography', 'language'], axis=1, inplace=True)
df_metadata.head()  # Looks good

,eadid,unit_title,unit_identifier,unique_language,unique_date,unique_geography
0,Coll-1064,"['Papers of Professor Walter Ledermann', '1 (3...","['Coll-1064', 'Coll-1064/1', 'Coll-1064/2', 'C...",[English],"[1937-01-01 - 1954-12-31, 1937-02-02 - 1938-02...","[Edinburgh (Scotland), London (England), Glena..."
1,Coll-31,['Drawings from the Office of Sir Rowand Ander...,"['Coll-31', 'Coll-31/1', 'Coll-31/1/1', 'Coll-...",[English],"[1814-01-01 - 1924-12-31, 1874-01-01 - 1905-12...",[]
2,Coll-51,['Papers of Sir Roderick Impey Murchison and h...,"['Coll-51', 'Coll-51/1', 'Coll-51/2', 'Coll-51...",[English],"[1771-01-01 - 1935-12-31, 1723-01-01 - 1935-12...","[Tarradale (Scotland), Europe, Scotland, Calcu..."
3,Coll-204,"['Lecture Notes of John Robison', 'Introductio...","['Coll-204', 'Coll-204/1', 'Coll-204/2', 'Coll...","[English, French, Latin, Russian]","[1779-01-01 - 1801-12-31, 1779-01-01 - 1801-12...","[Edinburgh (Scotland), Glasgow Lanarkshire Sco..."
4,Coll-206,['Records of the Wernerian Natural History Soc...,"['Coll-206', 'Coll-206/1', 'Coll-206/1/1', 'Co...",[English],"[1808-01-01 - 1858-12-31, 1808-01-12 - 1858-04...","[Edinburgh (Scotland), Freiburg im Breisgau (G..."


In [32]:
# df_metadata.to_csv("CRC_units-grouped-by-fonds_clean.csv")